In [1]:
pip install -U treelib

Note: you may need to restart the kernel to use updated packages.


In [2]:
from treelib import Node, Tree
import numpy as np

In [3]:
class Puzzle(object):
    def __init__(self, state):
        self.state = state
    
    def get_state(self):
        return np.copy(self.state)
    
    def set_state(self, state):
        self.state = state

In [4]:
global ROW
ROW = 3

global COL
COL = 4

start_state = Puzzle(
    np.array(
    [[4, 1, 2, 3],
    [5, 9, 6, 7],
    [np.nan, 8, 10, 11]])
)


goal_state = Puzzle(
    np.array(
    [[np.nan, 1, 2, 3],
    [4, 5, 6, 7],
    [8, 9, 10, 11]])
)

In [5]:
def get_inversion(self):
    prev_tiles = np.array([])
    inversion_count = 0
    
    for i in range(len(self.state)):
        for j in range(len(self.state[i])) :
            # Keep track of previous tiles
            prev_tiles = np.append(prev_tiles, self.state[i][j])
            
            # Subtract previous smaller from the total smaller to get the next smaller tiles count
            prev_smaller = np.count_nonzero(prev_tiles < self.state[i][j])
            total_smaller = np.count_nonzero(self.state < self.state[i][j])
            next_smaller = total_smaller - prev_smaller
            
            # Add the smaller count to the total inversion/parity
            inversion_count += next_smaller
    
    return inversion_count

In [6]:
def is_reachable(self, goal):
    # get start and goal state inversion
    ssi = self.get_inversion()
    gsi = goal.get_inversion()
    
    # if both are even or both are odd, return true (reachable)
    if (ssi % 2 == 0 and gsi % 2 == 0) or (ssi % 2 != 0 and gsi % 2 != 0):
        return True
    else:
        return False

Puzzle.get_inversion = get_inversion
Puzzle.is_reachable = is_reachable

In [7]:
def swap(state, src, dest):
    i, j = src
    k, l = dest
    swap = state[k][l]
    state[k][l] = state[i][j]
    state[i][j] = swap
    return state

In [8]:
def goal_test(state, goal):
    return ((state == goal) | (np.isnan(state) & np.isnan(goal))).all()

In [9]:
def dfs(start_state, goal_state, max_depth):
    tree = Tree() # maintaining state tree
    tag = 0 # tags for each node (name)
    i = 1 # used for iterating over tags
    prev_tag = 0 # parent tag
    curr_depth = 0
    
    # frontier/queue/stack
    stack = [(start_state.get_state(), 0, 0)] 
    
    # create start node in tree
    tree.create_node(prev_tag, prev_tag, data = start_state.get_state())
    
    while(len(stack) > 0 and curr_depth <= max_depth):
        # pop from the stack to get the current state and its tree-tag
        curr_state, prev_tag, curr_depth = stack.pop()
        
        # apply goal test
        result = goal_test(curr_state, goal_state.get_state())
        
        # if this state is the goal, return it
        if(result):
            print('Frontier:')
            for state in stack:
                print(state[1], end=" ")
            print('\n')
            
            print('Tree:')
            tree.show()
            
            return curr_depth
        
        # if this is the final depth, no need to expand further
        if(curr_depth == max_depth):
            continue

        # else expand this state
        empty = np.where(np.isnan(curr_state))
        empty_tile = empty[0][0], empty[1][0]
        r, c = empty_tile

        temp = Puzzle(np.copy(curr_state))

        # right
        if c + 1 < COL:
            temp.set_state(swap(np.copy(curr_state), empty_tile, (r, c + 1)))
            tree.create_node(tag + i, tag + i, data = temp.get_state(), parent = prev_tag)
            stack.append((temp.get_state(), tag + i, curr_depth + 1))
            i = i + 1

        # left
        if c - 1 >= 0:
            temp.set_state(swap(np.copy(curr_state), empty_tile, (r, c - 1)))
            tree.create_node(tag + i, tag + i, data = temp.get_state(), parent = prev_tag)
            stack.append((temp.get_state(), tag + i, curr_depth + 1))
            i = i + 1

        # down
        if r + 1 < ROW:
            temp.set_state(swap(np.copy(curr_state), empty_tile, (r + 1, c)))
            tree.create_node(tag + i, tag + i, data = temp.get_state(), parent = prev_tag)
            stack.append((temp.get_state(), tag + i, curr_depth + 1))
            i = i + 1

        # up
        if r - 1 >= 0:
            temp.set_state(swap(np.copy(curr_state), empty_tile, (r - 1, c)))
            tree.create_node(tag + i, tag + i, data = temp.get_state(), parent = prev_tag)
            stack.append((temp.get_state(), tag + i, curr_depth + 1))
            i = i + 1
        
        print('Frontier:')
        for state in stack:
            print(state[1], end=" ")
        print('\n')
        
    print('Tree:')  
    tree.show()
    print('---------------------')
    return False

In [10]:
def iterative_deepening_search(start_state, goal_state, max_depth):
    depth = 0
    solution = False
    
    while(not solution and depth <= max_depth):
        solution = dfs(start_state, goal_state, depth)
        depth = depth + 1
    
    return solution

In [11]:
if not start_state.is_reachable(goal_state):
    print('Goal is not reachable from this start state')
else:
    max_depth = 7
    result = iterative_deepening_search(start_state, goal_state, max_depth)
    if result:
        print('Goal found at depth', result)
    else:
        print('Goal was not found till depth', max_depth)

Tree:
0

---------------------
Frontier:
1 2 

Tree:
0
├── 1
└── 2

---------------------
Frontier:
1 2 

Frontier:
1 3 4 5 

Frontier:
6 7 8 

Tree:
0
├── 1
│   ├── 6
│   ├── 7
│   └── 8
└── 2
    ├── 3
    ├── 4
    └── 5

---------------------
Frontier:
1 2 

Frontier:
1 3 4 5 

Frontier:
1 3 4 6 7 

Frontier:
1 3 8 9 

Frontier:
1 10 11 12 13 

Frontier:
14 15 16 

Frontier:
14 15 17 18 19 20 

Frontier:
14 21 22 

Frontier:
23 24 25 

Tree:
0
├── 1
│   ├── 14
│   │   ├── 23
│   │   ├── 24
│   │   └── 25
│   ├── 15
│   │   ├── 21
│   │   └── 22
│   └── 16
│       ├── 17
│       ├── 18
│       ├── 19
│       └── 20
└── 2
    ├── 3
    │   ├── 10
    │   ├── 11
    │   ├── 12
    │   └── 13
    ├── 4
    │   ├── 8
    │   └── 9
    └── 5
        ├── 6
        └── 7

---------------------
Frontier:
1 2 

Frontier:
1 3 4 5 

Frontier:
1 3 4 6 7 

Frontier:
1 3 4 6 8 9 10 

Frontier:
1 3 4 11 12 13 

Frontier:
1 3 14 15 

Frontier:
1 3 14 16 17 18 

Frontier:
1 3 19 20 21 

Frontier:
1 